In [1]:
import pandas as pd
import numpy as np
from geopy import distance
from functools import reduce

In [2]:
# # merge 12 months data
# filepath10 = "JC-202110-citibike-tripdata.csv"
# filepath11 = "JC-202111-citibike-tripdata.csv"
# filepath12 = "JC-202112-citibike-tripdata.csv"
# filepath01 = "JC-202201-citibike-tripdata.csv"
# filepath02 = "JC-202202-citibike-tripdata.csv"
# filepath03 = "JC-202203-citibike-tripdata.csv"
# filepath04 = "JC-202204-citibike-tripdata.csv"
# filepath05 = "JC-202205-citibike-tripdata.csv"
# filepath06 = "JC-202206-citibike-tripdata.csv"
# filepath07 = "JC-202207-citbike-tripdata.csv"
# filepath08 = "JC-202208-citibike-tripdata.csv"
# filepath09 = "JC-202209-citibike-tripdata.csv"

# df10 = pd.read_csv(filepath10)
# df11 = pd.read_csv(filepath11)
# df12 = pd.read_csv(filepath12)
# df01 = pd.read_csv(filepath01)
# df02 = pd.read_csv(filepath02)
# df03 = pd.read_csv(filepath03)
# df04 = pd.read_csv(filepath04)
# df05 = pd.read_csv(filepath05)
# df06 = pd.read_csv(filepath06)
# df07 = pd.read_csv(filepath07)
# df08 = pd.read_csv(filepath08)
# df09 = pd.read_csv(filepath09)

# dfs = [df10, df11, df12, df01, df02, df03, df04, df05, df06, df07, df08, df09]

# df = pd.concat(dfs, ignore_index=True)

In [5]:
# merge 6 months data
filepath04 = "JC-202204-citibike-tripdata.csv"
filepath05 = "JC-202205-citibike-tripdata.csv"
filepath06 = "JC-202206-citibike-tripdata.csv"
filepath07 = "JC-202207-citbike-tripdata.csv"
filepath08 = "JC-202208-citibike-tripdata.csv"
filepath09 = "JC-202209-citibike-tripdata.csv"

df04 = pd.read_csv(filepath04)
df05 = pd.read_csv(filepath05)
df06 = pd.read_csv(filepath06)
df07 = pd.read_csv(filepath07)
df08 = pd.read_csv(filepath08)
df09 = pd.read_csv(filepath09)

dfs = [df04, df05, df06, df07, df08, df09]

df = pd.concat(dfs, ignore_index=True)

In [6]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,919C40A703A965D7,electric_bike,2022-04-15 15:02:20,2022-04-15 15:57:16,Pershing Field,JC024,Pershing Field,JC024,40.742677,-74.051789,40.742677,-74.051789,casual
1,3B40831921DAD6C1,classic_bike,2022-04-08 10:20:55,2022-04-08 10:25:29,JC Medical Center,JC011,Grand St,JC102,40.716540,-74.049638,40.715178,-74.037683,member
2,69C5C0766309F73D,electric_bike,2022-04-09 14:14:04,2022-04-09 14:18:51,Mama Johnson Field - 4 St & Jackson St,HB404,Southwest Park - Jackson St & Observer Hwy,HB401,40.743140,-74.040041,40.737551,-74.041664,member
3,304B82055A1D4A24,classic_bike,2022-04-05 19:43:37,2022-04-05 19:51:14,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735938,-74.030305,40.740802,-74.042521,casual
4,1CB760ECBCB08609,classic_bike,2022-04-17 11:25:26,2022-04-17 11:29:45,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,40.743140,-74.040041,40.743140,-74.040041,member


In [7]:
count_nan = df['end_lat'].isnull().sum()
count_nan

1316

In [8]:
# drop rows from df where end_lat is nan
df = df[df['end_lat'].notna()]
df.shape

(572973, 13)

In [9]:
# check that nan values are gone
count_nan = df['end_lat'].isnull().sum()
count_nan

0

In [10]:
df.shape

(572973, 13)

In [11]:
# Calculating ride duration

df['ended_at'] = df['ended_at'].astype('datetime64[ns]')
df['started_at'] = df['started_at'].astype('datetime64[ns]')
df['ride_duration_timedelta'] = df['ended_at'] - df['started_at']

ride_hrs = []

for i in df['ride_duration_timedelta']:
    hrs = i.total_seconds()/3600
    ride_hrs.append(hrs)

df['ride_duration_hrs'] = ride_hrs

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration_timedelta,ride_duration_hrs
0,919C40A703A965D7,electric_bike,2022-04-15 15:02:20,2022-04-15 15:57:16,Pershing Field,JC024,Pershing Field,JC024,40.742677,-74.051789,40.742677,-74.051789,casual,0 days 00:54:56,0.915556
1,3B40831921DAD6C1,classic_bike,2022-04-08 10:20:55,2022-04-08 10:25:29,JC Medical Center,JC011,Grand St,JC102,40.716540,-74.049638,40.715178,-74.037683,member,0 days 00:04:34,0.076111
2,69C5C0766309F73D,electric_bike,2022-04-09 14:14:04,2022-04-09 14:18:51,Mama Johnson Field - 4 St & Jackson St,HB404,Southwest Park - Jackson St & Observer Hwy,HB401,40.743140,-74.040041,40.737551,-74.041664,member,0 days 00:04:47,0.079722
3,304B82055A1D4A24,classic_bike,2022-04-05 19:43:37,2022-04-05 19:51:14,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735938,-74.030305,40.740802,-74.042521,casual,0 days 00:07:37,0.126944
4,1CB760ECBCB08609,classic_bike,2022-04-17 11:25:26,2022-04-17 11:29:45,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,40.743140,-74.040041,40.743140,-74.040041,member,0 days 00:04:19,0.071944


In [12]:
df.describe()

,start_lat,start_lng,end_lat,end_lng,ride_duration_timedelta,ride_duration_hrs
count,572973.000000,572973.000000,572973.000000,572973.000000,572973,572973.000000
mean,40.731909,-74.040270,40.731859,-74.040040,0 days 00:15:22.698036382,0.256305
std,0.012199,0.012172,0.012330,0.012184,0 days 01:37:14.033404912,1.620565
min,40.706531,-74.086814,40.640000,-74.140000,-1 days +23:59:56,-0.001111
25%,40.721124,-74.045953,40.719586,-74.045953,0 days 00:04:46,0.079444
50%,40.733670,-74.037683,40.733670,-74.037683,0 days 00:07:45,0.129167
75%,40.740973,-74.030970,40.740973,-74.030970,0 days 00:13:32,0.225556
max,40.754992,-74.023587,40.849972,-73.888271,19 days 19:41:30,475.691667


In [13]:
distance_miles = []

for i, row in df.iterrows():
    
    start = (df['start_lat'][i], df['start_lng'][i])
    end = (df['end_lat'][i], df['end_lng'][i])
    
#     print(f'start {start}, end {end}')
    try:    
        dist = distance.distance(start, end).miles
    except:
        print("There was a problem")
    
    distance_miles.append(dist)
    
distance_miles

[0.0,
 0.6346513481332742,
 0.39492516383311044,
 0.723728882142965,
 0.0,
 0.0,
 0.0,
 0.6428095425298869,
 0.8380579917604118,
 0.6393829834401678,
 0.8758472195924946,
 0.8758472195924946,
 0.8758472195924946,
 0.8758472195924946,
 0.8758472195924946,
 0.38270067969507154,
 0.8137001878093334,
 0.35167359430900474,
 0.637488478939282,
 0.637488478939282,
 0.1429885972977985,
 0.1429885972977985,
 0.1429885972977985,
 1.3499110783207262,
 0.7378103508812102,
 1.1128302876275606,
 1.2157440950539258,
 1.1133775697222275,
 0.5558722389243468,
 0.7001990965724023,
 0.9310908284151446,
 0.2072838071702382,
 1.9697065905973303,
 0.5549408739035734,
 0.5549408739035734,
 0.5549408739035734,
 0.6511150937102979,
 0.5225566715393649,
 0.5225566715393649,
 0.5225566715393649,
 0.4509425450629926,
 0.4509425450629926,
 0.4509425450629926,
 0.19885087287299177,
 0.8308905640676164,
 0.3025797449843733,
 1.219718754189552,
 0.567293075442022,
 0.5549408739035734,
 0.7328336069709889,
 1.08128612

In [14]:
df['ride_distance_miles'] = distance_miles

In [ ]:
# # save to csv 12 monts data
# df.to_csv('202110-202209_clean.csv')

In [15]:
# save to csv 6 months data
df.to_csv('202204-202209_clean.csv')